In [1]:
import os
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
PATH = '/home/kai/data/kaggle/homecredit/'

In [2]:
train = pd.read_pickle(PATH + 'test/train_base.pkl')
test = pd.read_pickle(PATH + 'test/test_base.pkl')



(307511, 329) (48744, 328)
(356255, 818)
(356255, 116)
(356255, 502)
(356255, 60)
(356255, 554)
(307511, 1146) (48744, 1145)
(307511, 1261) (48744, 1260)
(307511, 1762) (48744, 1761)
(307511, 1821) (48744, 1820)
(307511, 2374) (48744, 2373)


In [3]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [4]:
df = pd.concat([train, test], axis = 0)

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [5]:
install.columns
install_amt = [col for col in df.columns if 'AMT' in col and 'install' in col and 'divide' not in col]

In [7]:
train_amt = ['AMT_CREDIT','AMT_ANNUITY']

In [8]:
for i in train_amt:
    for j in install_amt:
        df[ratio_name(i,j)] = ratio(df,i,j)

In [9]:
df.shape

(356255, 2412)

In [10]:
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
test.drop('TARGET', inplace = True, axis = 1)
print(train.shape, test.shape)

(307511, 2412) (48744, 2411)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# To file

In [11]:
train.to_pickle(PATH +'test/train_install1.pkl')
test.to_pickle(PATH +'test/test_install1.pkl')